In [ ]:
import theano
import theano.tensor as T

x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)

In [1]:
import scipy.optimize as opt
from converge import Converge
from numpy.linalg import inv

def minimize(f, x, g, H, epsilon = 0.0000001, maxsteps = 100):
    converge = Converge(f, epsilon, maxsteps)
    
    while not converge.done(x):
        d = -inv(H(x)).dot(g(x))
        x += d * opt.minimize(lambda eta: f(x + eta * d), 1).x
    return x

In [7]:
import numpy as np
from scipy import stats
import derive
reload(derive)
import scipy.optimize as opt
import sympy as sm

fun = lambda x: (3 * x[0] + x[1]) ** 2
point = [0.1, 0.3]

print derive.grad(fun)(point)
print derive.hess(fun)(point)

x = sm.symbols(' '.join(['x%s' % i for i in xrange(2)]))
fun = (3 * x[0] + x[1]) ** 2
print derive.grad(fun)(point)

pdf = stats.multivariate_normal((23, 7), 4).pdf
fun = lambda x: -pdf(x)
x = [10, 10]
print minimize(fun, x, derive.grad(fun), derive.hess(fun)) 
print opt.minimize(fun, x).x

[ 3.6       1.200006]
[[ 17.99994875   5.99995054]
 [  5.99995054   2.00008066]]
[3.60000000000000, 1.20000000000000]
[ -334167.8604396  -1215755.63522998]
[10 10]
